## FEATURE EXTRACTION AND DATA PREPROCESSING

In [1]:
import os
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.utils import resample

In [2]:
def extract_features(file_path):
    y, sr = librosa.load(file_path, sr=None)
    
    # Trim silence from the audio
    y, _ = librosa.effects.trim(y, top_db=20)
    
    # Extract features
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    rms_energy = np.mean(librosa.feature.rms(y=y))
    
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=21)
    mfccs_mean = np.mean(mfccs, axis=1)
    
    features = [zcr, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_rolloff, rms_energy] + list(mfccs_mean)
    return features

In [3]:
def process_folder(folder_path, label):
    data = []
    for file_name in tqdm(os.listdir(folder_path), desc=f"Processing {label} files"):
        if file_name.endswith('.wav'):
            file_path = os.path.join(folder_path, file_name)
            features = extract_features(file_path)
            features.append(file_name)  
            features.append(label) 
            data.append(features)
    return data

In [4]:
cough_folder_path = "H:/My Drive/COUGH"
non_cough_folder_path = "H:/My Drive/NOT_COUGH"

In [5]:
cough_data = process_folder(cough_folder_path, 'cough')
non_cough_data = process_folder(non_cough_folder_path, 'non_cough')

Processing non_cough files: 100%|████████████████████████████████████████████████████| 431/431 [03:20<00:00,  2.15it/s]


In [6]:
combine_data = cough_data + non_cough_data

In [7]:
columns = ['zcr', 'spectral_centroid', 'spectral_bandwidth', 'spectral_contrast', 'spectral_rolloff', 'rms_energy'] + [f'mfcc_{i}' for i in range(1, 22)] + ['file_name', 'label']

In [8]:
df = pd.DataFrame(combine_data, columns=columns)

In [9]:
df = df.drop(['file_name'], axis=1)

In [10]:
df.to_csv('H:/My Drive/Task 2 - Feature Extarction/CNN Model 3/feature_extraction.csv', index=False)

In [11]:
df_features = pd.read_csv('H:/My Drive/Task 2 - Feature Extarction/CNN Model 3/feature_extraction.csv')

In [12]:
df_majority = df_features[df_features['label'] == 'non_cough']
df_minority = df_features[df_features['label'] == 'cough']
df_majority_balanced = resample(df_majority, replace=False, n_samples=len(df_minority), random_state=42)
df_balanced = pd.concat([df_majority_balanced, df_minority])

In [13]:
df_balanced.to_csv('H:/My Drive/Task 2 - Feature Extarction/CNN Model 3/feature_extraction.csv', index=False)

In [14]:
print('Total number of examples:', len(df_balanced))
print('Number of cough examples:', len(df_balanced[df_balanced['label'] == 'cough']))
print('Number of not cough examples:', len(df_balanced[df_balanced['label'] == 'non_cough']))

Total number of examples: 862
Number of cough examples: 431
Number of not cough examples: 431


In [15]:
df = pd.read_csv("feature_extraction.csv")

In [16]:
df

,zcr,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_rolloff,rms_energy,mfcc_1,mfcc_2,mfcc_3,mfcc_4,...,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,label
0,0.029213,1112.871564,1604.056017,24.892593,1760.744265,0.081122,-448.81583,122.14291,-30.411432,-21.905825,...,-17.041662,-19.493896,-17.205946,-9.201260,-0.198663,5.860787,7.860169,7.522961,8.683452,non_cough
1,0.012457,1050.948040,2174.865326,15.973477,1694.400239,0.040163,-412.30994,130.85251,16.608328,69.699390,...,-8.629910,-6.937414,-1.147614,-5.570846,0.199644,1.057782,-6.458854,0.350183,-0.142641,non_cough
2,0.100690,3195.011947,3049.857138,16.933318,5852.706205,0.109561,-293.02173,150.12302,-19.515726,-0.561357,...,5.950315,-3.206813,-0.501098,-3.582239,-4.231604,2.062259,2.788978,-0.896737,-0.267438,non_cough
3,0.001611,1067.626882,2618.348851,12.851745,1894.921875,0.250158,-320.02070,119.57776,6.242011,41.371864,...,11.427472,10.879184,10.726569,9.264940,5.624715,2.997437,5.239369,6.776089,3.964313,non_cough
4,0.044128,2770.453081,2867.777937,19.252790,5707.644653,0.161779,-195.62204,129.60016,-58.906826,52.263382,...,7.825397,-2.478972,6.249976,9.515294,-5.330663,1.446208,-8.028227,-5.254991,-3.608084,non_cough
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,0.079220,2693.482587,2599.789602,17.428186,4800.445662,0.040164,-300.02795,155.08849,-66.840660,20.671022,...,-8.014601,6.603203,-1.622552,-0.095497,5.154353,-5.718300,1.815212,-5.435640,-2.056177,cough
858,0.086063,4237.359232,4647.301584,17.323820,8733.250537,0.014105,-427.93470,102.49376,-26.879637,7.111455,...,-19.243622,-7.554722,-6.590545,-4.727139,-4.538400,-4.069426,-5.873992,-1.550436,-5.683088,cough
859,0.024711,1195.888758,1972.589472,17.095105,2294.282131,0.078553,-306.71332,169.64798,5.926561,-17.236458,...,4.376550,-4.724573,4.331160,9.613692,1.660168,1.348640,6.709805,2.372028,-0.943119,cough
860,0.108596,3871.364219,4093.153195,17.292438,7456.062180,0.011830,-444.11957,127.43482,-37.609917,35.145657,...,2.233887,-0.598832,10.553861,-9.460074,-0.121605,4.381053,-6.817329,2.434754,-3.922333,cough


## RANDOM FOREST CLASSIFIER

In [17]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

In [19]:
df = pd.read_csv('H:/My Drive/Task 2 - Feature Extarction/CNN Model 3/feature_extraction.csv')

In [20]:
X = df.drop(['label'], axis=1)
y = df['label']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

In [23]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, random_state=42)

In [25]:
y_pred = rf.predict(X_test)

In [26]:
acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, average='macro')
r = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

In [27]:
print("Accuracy: {:.2f}".format(acc))
print("Precision: {:.2f}".format(p))
print("Recall: {:.2f}".format(r))
print("F1 Score: {:.2f}".format(f1))

Accuracy: 0.85
Precision: 0.85
Recall: 0.85
F1 Score: 0.85


In [28]:
joblib.dump(rf, 'H:/My Drive/Task 2 - Feature Extarction/CNN Model 3/model_randon_forest.pkl')

['H:/My Drive/Task 2 - Feature Extarction/CNN Model 3/model_randon_forest.pkl']

### PREDICTION

In [31]:
import joblib

model = joblib.load('model_randon_forest.pkl')

def predict(file_path):
    features = extract_features(file_path)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    return prediction[0]

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
prediction = predict(new_audio_signal)
print("The prediction for ",new_audio_signal," is ",prediction)

The prediction for  H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav  is  non_cough


C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## SUPPORT VECTOR MACHINE

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

In [33]:
df = pd.read_csv('feature_extraction.csv')

In [34]:
df

,zcr,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_rolloff,rms_energy,mfcc_1,mfcc_2,mfcc_3,mfcc_4,...,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,label
0,0.029213,1112.871564,1604.056017,24.892593,1760.744265,0.081122,-448.81583,122.14291,-30.411432,-21.905825,...,-17.041662,-19.493896,-17.205946,-9.201260,-0.198663,5.860787,7.860169,7.522961,8.683452,non_cough
1,0.012457,1050.948040,2174.865326,15.973477,1694.400239,0.040163,-412.30994,130.85251,16.608328,69.699390,...,-8.629910,-6.937414,-1.147614,-5.570846,0.199644,1.057782,-6.458854,0.350183,-0.142641,non_cough
2,0.100690,3195.011947,3049.857138,16.933318,5852.706205,0.109561,-293.02173,150.12302,-19.515726,-0.561357,...,5.950315,-3.206813,-0.501098,-3.582239,-4.231604,2.062259,2.788978,-0.896737,-0.267438,non_cough
3,0.001611,1067.626882,2618.348851,12.851745,1894.921875,0.250158,-320.02070,119.57776,6.242011,41.371864,...,11.427472,10.879184,10.726569,9.264940,5.624715,2.997437,5.239369,6.776089,3.964313,non_cough
4,0.044128,2770.453081,2867.777937,19.252790,5707.644653,0.161779,-195.62204,129.60016,-58.906826,52.263382,...,7.825397,-2.478972,6.249976,9.515294,-5.330663,1.446208,-8.028227,-5.254991,-3.608084,non_cough
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,0.079220,2693.482587,2599.789602,17.428186,4800.445662,0.040164,-300.02795,155.08849,-66.840660,20.671022,...,-8.014601,6.603203,-1.622552,-0.095497,5.154353,-5.718300,1.815212,-5.435640,-2.056177,cough
858,0.086063,4237.359232,4647.301584,17.323820,8733.250537,0.014105,-427.93470,102.49376,-26.879637,7.111455,...,-19.243622,-7.554722,-6.590545,-4.727139,-4.538400,-4.069426,-5.873992,-1.550436,-5.683088,cough
859,0.024711,1195.888758,1972.589472,17.095105,2294.282131,0.078553,-306.71332,169.64798,5.926561,-17.236458,...,4.376550,-4.724573,4.331160,9.613692,1.660168,1.348640,6.709805,2.372028,-0.943119,cough
860,0.108596,3871.364219,4093.153195,17.292438,7456.062180,0.011830,-444.11957,127.43482,-37.609917,35.145657,...,2.233887,-0.598832,10.553861,-9.460074,-0.121605,4.381053,-6.817329,2.434754,-3.922333,cough


In [35]:
X = df.drop(['label'], axis=1)
y = df['label']

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)

SVC(kernel='linear', random_state=42)

In [38]:
y_pred = svm_model.predict(X_test)

In [39]:
acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, average='macro')
r = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

In [40]:
print("Accuracy: {:.2f}".format(acc))
print("Precision: {:.2f}".format(p))
print("Recall: {:.2f}".format(r))
print("F1 Score: {:.2f}".format(f1))

Accuracy: 0.71
Precision: 0.71
Recall: 0.71
F1 Score: 0.71


In [41]:
joblib.dump(svm_model, 'svm_model.pkl')

['svm_model.pkl']

### PREDICTION

In [42]:
import joblib

model = joblib.load('svm_model.pkl')

def predict(file_path):
    features = extract_features(file_path)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    return prediction[0]

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
prediction = predict(new_audio_signal)
print("The prediction for ",new_audio_signal," is ",prediction)

The prediction for  H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav  is  non_cough


C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


## GRADIENT BOOSTING CLASSIFIER

In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

In [44]:
df = pd.read_csv('feature_extraction.csv')

In [45]:
df

,zcr,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_rolloff,rms_energy,mfcc_1,mfcc_2,mfcc_3,mfcc_4,...,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,label
0,0.029213,1112.871564,1604.056017,24.892593,1760.744265,0.081122,-448.81583,122.14291,-30.411432,-21.905825,...,-17.041662,-19.493896,-17.205946,-9.201260,-0.198663,5.860787,7.860169,7.522961,8.683452,non_cough
1,0.012457,1050.948040,2174.865326,15.973477,1694.400239,0.040163,-412.30994,130.85251,16.608328,69.699390,...,-8.629910,-6.937414,-1.147614,-5.570846,0.199644,1.057782,-6.458854,0.350183,-0.142641,non_cough
2,0.100690,3195.011947,3049.857138,16.933318,5852.706205,0.109561,-293.02173,150.12302,-19.515726,-0.561357,...,5.950315,-3.206813,-0.501098,-3.582239,-4.231604,2.062259,2.788978,-0.896737,-0.267438,non_cough
3,0.001611,1067.626882,2618.348851,12.851745,1894.921875,0.250158,-320.02070,119.57776,6.242011,41.371864,...,11.427472,10.879184,10.726569,9.264940,5.624715,2.997437,5.239369,6.776089,3.964313,non_cough
4,0.044128,2770.453081,2867.777937,19.252790,5707.644653,0.161779,-195.62204,129.60016,-58.906826,52.263382,...,7.825397,-2.478972,6.249976,9.515294,-5.330663,1.446208,-8.028227,-5.254991,-3.608084,non_cough
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,0.079220,2693.482587,2599.789602,17.428186,4800.445662,0.040164,-300.02795,155.08849,-66.840660,20.671022,...,-8.014601,6.603203,-1.622552,-0.095497,5.154353,-5.718300,1.815212,-5.435640,-2.056177,cough
858,0.086063,4237.359232,4647.301584,17.323820,8733.250537,0.014105,-427.93470,102.49376,-26.879637,7.111455,...,-19.243622,-7.554722,-6.590545,-4.727139,-4.538400,-4.069426,-5.873992,-1.550436,-5.683088,cough
859,0.024711,1195.888758,1972.589472,17.095105,2294.282131,0.078553,-306.71332,169.64798,5.926561,-17.236458,...,4.376550,-4.724573,4.331160,9.613692,1.660168,1.348640,6.709805,2.372028,-0.943119,cough
860,0.108596,3871.364219,4093.153195,17.292438,7456.062180,0.011830,-444.11957,127.43482,-37.609917,35.145657,...,2.233887,-0.598832,10.553861,-9.460074,-0.121605,4.381053,-6.817329,2.434754,-3.922333,cough


In [46]:
X = df.drop(['label'], axis=1)
y = df['label']

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [48]:
X_train.shape

(689, 27)

In [49]:
X_test.shape

(173, 27)

In [50]:
gbc_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbc_model.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [51]:
y_pred = gbc_model.predict(X_test)

In [52]:
acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, average='macro')
r = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

In [53]:
print("Accuracy: {:.2f}".format(acc))
print("Precision: {:.2f}".format(p))
print("Recall: {:.2f}".format(r))
print("F1 Score: {:.2f}".format(f1))

Accuracy: 0.86
Precision: 0.86
Recall: 0.86
F1 Score: 0.86


In [54]:
joblib.dump(gbc_model, 'gbc_model.pkl')

['gbc_model.pkl']

### PREDICTION

In [55]:
import joblib

model = joblib.load('gbc_model.pkl')

def predict(file_path):
    features = extract_features(file_path)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    return prediction[0]

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
prediction = predict(new_audio_signal)
print("The prediction for ",new_audio_signal," is ",prediction)

The prediction for  H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav  is  non_cough


C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


## SUPPORT VECTOR MACHINE HYPERPARAMETER TUNING

In [122]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import joblib

In [123]:
df = pd.read_csv('feature_extraction.csv')

In [124]:
X = df.drop(['label'], axis=1)
y = df['label']

In [125]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [126]:
X_train.shape

(689, 27)

In [127]:
X_test.shape

(173, 27)

In [128]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [129]:
param_grid = {
    'C': [0.1, 1, 10, 100], 
    'gamma': [0.001, 0.01, 0.1, 1], 
    'kernel': ['linear', 'rbf', 'poly']
}

In [130]:
grid_search = GridSearchCV(SVC(random_state=42), param_grid, cv=3, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 48 candidates, totalling 144 fits


GridSearchCV(cv=3, estimator=SVC(random_state=42), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [0.001, 0.01, 0.1, 1],
                         'kernel': ['linear', 'rbf', 'poly']},
             scoring='accuracy', verbose=2)

In [131]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}


In [132]:
svm_model_best = grid_search.best_estimator_
svm_model_best.fit(X_train, y_train)

SVC(C=1, gamma=0.1, random_state=42)

In [133]:
y_pred = svm_model_best.predict(X_test_scaled)

C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [134]:
acc = accuracy_score(y_test, y_pred)
p = precision_score(y_test, y_pred, average='weighted')
r = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [135]:
print("Improved  Support Vector Machine Evaluation : \n")
print("Accuracy: {:.2f}".format(acc))
print("Precision: {:.2f}".format(p))
print("Recall: {:.2f}".format(r))
print("F1 Score: {:.2f}".format(f1))

Improved  Support Vector Machine Evaluation : 

Accuracy: 0.49
Precision: 0.24
Recall: 0.49
F1 Score: 0.32


In [136]:
joblib.dump(svm_model_best, 'svm_model_best.pkl')

['svm_model_best.pkl']

### PREDICTION

In [137]:
import joblib

model = joblib.load('svm_model_best.pkl')

def predict(file_path):
    features = extract_features(file_path)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    return prediction[0]

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
prediction = predict(new_audio_signal)
print("The prediction for ",new_audio_signal," is ",prediction)

The prediction for  H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav  is  non_cough


C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


## RANDOM FOREST CLASSIFIER HYPERPARAMETER TUNING

In [138]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import joblib

In [139]:
df = pd.read_csv('feature_extraction.csv')

In [140]:
X = df.drop(['label'], axis=1)
y = df['label']

In [141]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [142]:
X_train.shape

(689, 27)

In [143]:
X_test.shape

(173, 27)

In [144]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [145]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

In [146]:
rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, n_iter=20, cv=5, scoring='accuracy', random_state=42, n_jobs=-1)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_depth': [None, 10, 20, 30],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [50, 100, 200]},
                   random_state=42, scoring='accuracy')

In [147]:
best_params = random_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 30}


In [148]:
best_rf_model = random_search.best_estimator_
y_pred = best_rf_model.predict(X_test_scaled)

C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [149]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [150]:
print("Improved Random Forest Classifier Evaluation:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Improved Random Forest Classifier Evaluation:
Accuracy: 0.4913294797687861
Precision: 0.241404657689866
Recall: 0.4913294797687861
F1 Score: 0.3237442308554017


In [151]:
joblib.dump(best_rf_model, 'model_random_forest_best.pkl')

['model_random_forest_best.pkl']

### PREDICTION

In [152]:
import joblib

model = joblib.load('model_random_forest_best.pkl')

def predict(file_path):
    features = extract_features(file_path)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    return prediction[0]

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
prediction = predict(new_audio_signal)
print("The prediction for ",new_audio_signal," is ",prediction)

The prediction for  H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav  is  non_cough


C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## GRADIENT BOOSTING CLASSIFIER HYPERPARAMETER TUNING

In [153]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler

In [154]:
df = pd.read_csv('feature_extraction.csv')

In [155]:
X = df.drop(['label'], axis=1)
y = df['label']

In [156]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [157]:
X_train.shape

(689, 27)

In [158]:
X_test.shape

(173, 27)

In [159]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [160]:
param_grid = {
    'n_estimators': [50, 100, 200],             
    'learning_rate': [0.1, 0.05, 0.01],         
    'max_depth': [3, 5, 7],                     
}

In [161]:
grid_search = GridSearchCV(GradientBoostingClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', verbose=2)
grid_search.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   1.2s
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   1.2s
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   1.1s
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   1.1s
[CV] END ....learning_rate=0.1, max_depth=3, n_estimators=50; total time=   1.1s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   2.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   2.3s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   2.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   2.2s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=100; total time=   2.1s
[CV] END ...learning_rate=0.1, max_depth=3, n_estimators=200; total time=   4.1s
[CV] END ...learning_rate=0.1, max_depth=3, n_e

[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   4.7s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   4.4s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   4.7s
[CV] END ..learning_rate=0.01, max_depth=3, n_estimators=200; total time=   4.8s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   1.8s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   1.8s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   1.7s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   1.8s
[CV] END ...learning_rate=0.01, max_depth=5, n_estimators=50; total time=   1.8s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   3.9s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   3.8s
[CV] END ..learning_rate=0.01, max_depth=5, n_estimators=100; total time=   3.8s
[CV] END ..learning_rate=0.0

GridSearchCV(cv=5, estimator=GradientBoostingClassifier(random_state=42),
             param_grid={'learning_rate': [0.1, 0.05, 0.01],
                         'max_depth': [3, 5, 7],
                         'n_estimators': [50, 100, 200]},
             scoring='accuracy', verbose=2)

In [162]:
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

Best Parameters: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200}


In [163]:
model_gbc_best = grid_search.best_estimator_
model_gbc_best.fit(X_train_scaled, y_train)

GradientBoostingClassifier(max_depth=7, n_estimators=200, random_state=42)

In [164]:
y_pred = model_gbc_best.predict(X_test_scaled)

In [165]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

In [166]:
print("Improved Gradient Boosting Classifier Evaluation:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Improved Gradient Boosting Classifier Evaluation:
Accuracy: 0.8323699421965318
Precision: 0.836300048727405
Recall: 0.8323699421965318
F1 Score: 0.8320671280553821


In [167]:
joblib.dump(best_rf_model, 'model_gbc_best.pkl')

['model_gbc_best.pkl']

### PREDICTION

In [168]:
import joblib

model = joblib.load('model_gbc_best.pkl')

def predict(file_path):
    features = extract_features(file_path)
    features = np.array(features).reshape(1, -1)
    prediction = model.predict(features)
    return prediction[0]

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
prediction = predict(new_audio_signal)
print("The prediction for ",new_audio_signal," is ",prediction)

The prediction for  H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav  is  non_cough


C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


## CONVULUTIONAL NEURAL NETWORKS - 1

In [244]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [245]:
df = pd.read_csv('feature_extraction.csv')

In [246]:
X = df.drop(['label'], axis=1)
y = df['label']

In [247]:
y = y.map({'cough': 1, 'non_cough': 0})

In [248]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [249]:
X_train.shape

(689, 27)

In [250]:
X_test.shape

(173, 27)

In [251]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [252]:
X_train_scaled = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [253]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization

In [254]:
def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Conv1D(128, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Conv1D(256, kernel_size=3, activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    return model

In [255]:
input_shape = (X_train_scaled.shape[1], 1)
model = create_cnn_model(input_shape)

C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [256]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test), epochs=10, batch_size=128)

Epoch 1/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 186ms/step - accuracy: 0.5872 - loss: 0.8668 - val_accuracy: 0.5434 - val_loss: 0.6653
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.7721 - loss: 0.4996 - val_accuracy: 0.8150 - val_loss: 0.6600
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.7672 - loss: 0.5181 - val_accuracy: 0.7572 - val_loss: 0.6607
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.7857 - loss: 0.4573 - val_accuracy: 0.7110 - val_loss: 0.6463
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.8167 - loss: 0.4159 - val_accuracy: 0.6474 - val_loss: 0.6358
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.8339 - loss: 0.3931 - val_accuracy: 0.6012 - val_loss: 0.6332
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.8193 - loss: 0.3932 - val_accuracy: 0.5896 - val_loss: 0.6359
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.8440 - loss: 0.3908 - val_accuracy: 0.5607 - val_loss: 0.6446

In [257]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_9 (Conv1D)                    │ (None, 25, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_6                │ (None, 25, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_9 (MaxPooling1D)       │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_10 (Conv1D)                   │ (None, 10, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_7                │ (None, 10, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_10 (MaxPooling1D)      │ (None, 5, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_12 (Dropout)                 │ (None, 5, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_11 (Conv1D)                   │ (None, 3, 256)              │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 3, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_11 (MaxPooling1D)      │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_13 (Dropout)                 │ (None, 1, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_14 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 572,293 (2.18 MB)

 Trainable params: 190,465 (744.00 KB)

 Non-trainable params: 896 (3.50 KB)

 Optimizer params: 380,932 (1.45 MB)

In [258]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [259]:
y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")

6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step


In [260]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [261]:
print("Convolutional Neural Networks Evaluation :")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Convolutional Neural Networks Evaluation :
Accuracy: 0.5375722543352601
Precision: 0.5238095238095238
Recall: 1.0
F1 Score: 0.6875000000000001


In [262]:
model.save('cnn_cough_detection_model.h5')

### PREDICTION

In [264]:
from tensorflow.keras.models import load_model

model = load_model('cnn_cough_detection_model.h5')

def predict(audio_file):
    features = extract_features(audio_file)
    
    features_df = pd.DataFrame([features])
    
    features_scaled = scaler.transform(features_df)
    
    features_scaled = features_scaled.reshape(features_scaled.shape[0], features_scaled.shape[1], 1)
    
    prediction = (model.predict(features_scaled) > 0.5).astype("int32")
    
    label = 'cough' if prediction[0][0] == 1 else 'non cough'
    
    return label

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
prediction = predict(new_audio_signal)
print("The prediction for ",new_audio_signal," is ",prediction)

C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 455ms/step
The prediction for  H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav  is  cough


## DENSE NEURAL NETWORKS - 1

In [265]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras import models, layers, callbacks

In [266]:
df = pd.read_csv("feature_extraction.csv")

In [267]:
df

,zcr,spectral_centroid,spectral_bandwidth,spectral_contrast,spectral_rolloff,rms_energy,mfcc_1,mfcc_2,mfcc_3,mfcc_4,...,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,mfcc_21,label
0,0.029213,1112.871564,1604.056017,24.892593,1760.744265,0.081122,-448.81583,122.14291,-30.411432,-21.905825,...,-17.041662,-19.493896,-17.205946,-9.201260,-0.198663,5.860787,7.860169,7.522961,8.683452,non_cough
1,0.012457,1050.948040,2174.865326,15.973477,1694.400239,0.040163,-412.30994,130.85251,16.608328,69.699390,...,-8.629910,-6.937414,-1.147614,-5.570846,0.199644,1.057782,-6.458854,0.350183,-0.142641,non_cough
2,0.100690,3195.011947,3049.857138,16.933318,5852.706205,0.109561,-293.02173,150.12302,-19.515726,-0.561357,...,5.950315,-3.206813,-0.501098,-3.582239,-4.231604,2.062259,2.788978,-0.896737,-0.267438,non_cough
3,0.001611,1067.626882,2618.348851,12.851745,1894.921875,0.250158,-320.02070,119.57776,6.242011,41.371864,...,11.427472,10.879184,10.726569,9.264940,5.624715,2.997437,5.239369,6.776089,3.964313,non_cough
4,0.044128,2770.453081,2867.777937,19.252790,5707.644653,0.161779,-195.62204,129.60016,-58.906826,52.263382,...,7.825397,-2.478972,6.249976,9.515294,-5.330663,1.446208,-8.028227,-5.254991,-3.608084,non_cough
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,0.079220,2693.482587,2599.789602,17.428186,4800.445662,0.040164,-300.02795,155.08849,-66.840660,20.671022,...,-8.014601,6.603203,-1.622552,-0.095497,5.154353,-5.718300,1.815212,-5.435640,-2.056177,cough
858,0.086063,4237.359232,4647.301584,17.323820,8733.250537,0.014105,-427.93470,102.49376,-26.879637,7.111455,...,-19.243622,-7.554722,-6.590545,-4.727139,-4.538400,-4.069426,-5.873992,-1.550436,-5.683088,cough
859,0.024711,1195.888758,1972.589472,17.095105,2294.282131,0.078553,-306.71332,169.64798,5.926561,-17.236458,...,4.376550,-4.724573,4.331160,9.613692,1.660168,1.348640,6.709805,2.372028,-0.943119,cough
860,0.108596,3871.364219,4093.153195,17.292438,7456.062180,0.011830,-444.11957,127.43482,-37.609917,35.145657,...,2.233887,-0.598832,10.553861,-9.460074,-0.121605,4.381053,-6.817329,2.434754,-3.922333,cough


In [268]:
genre_list = df.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [269]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(df.iloc[:, :-1], dtype=float))

In [270]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [271]:
model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [272]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.weights.h5")
print("Saved model to disk")

Saved model to disk


In [273]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [274]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                     │ (None, 256)                 │           7,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_15 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_16 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_17 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 48,450 (189.26 KB)

 Trainable params: 48,450 (189.26 KB)

 Non-trainable params: 0 (0.00 B)

In [275]:
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [281]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=[early_stopping])

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.8374 - loss: 0.3379 - val_accuracy: 0.8276 - val_loss: 0.3932
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8722 - loss: 0.3127 - val_accuracy: 0.8362 - val_loss: 0.3960
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.8586 - loss: 0.3263 - val_accuracy: 0.8276 - val_loss: 0.3987
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.8790 - loss: 0.2793 - val_accuracy: 0.8276 - val_loss: 0.4036
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.8524 - loss: 0.3128 - val_accuracy: 0.8276 - val_loss: 0.4067


In [282]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [283]:
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes)
recall = recall_score(y_test, y_pred_classes)
f1 = f1_score(y_test, y_pred_classes)

In [284]:
print("Dense Neural Network Evaluation:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Dense Neural Network Evaluation:
Accuracy: 0.8
Precision: 0.8201438848920863
Recall: 0.7808219178082192
F1 Score: 0.7999999999999999


In [285]:
model.save('dense_nn_cough_detection_model.h5')

### PREDICTION

In [287]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
audio_features = extract_features(new_audio_signal)

model = load_model('dense_nn_cough_detection_model.h5')

scaler = StandardScaler()  
audio_features_scaled = scaler.fit_transform(np.array(audio_features).reshape(1, -1))

prediction = model.predict(audio_features_scaled)

if prediction[0][0] < 0.5:
    print("The audio file does not contain a cough")
else:
    print("The audio file contains a cough.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step
The audio file contains a cough.


## CONVOLUTIONAL NEURAL NETOWKS - 2

In [288]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import models, layers
from tensorflow.keras.utils import to_categorical

In [289]:
df = pd.read_csv("feature_extraction.csv")

In [290]:
X = df.drop(columns=['label']).values
y = df['label'].values

In [291]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [292]:
X_train, X_val, y_train, y_val = train_test_split(X, y_categorical, test_size=0.33, random_state=42)

In [293]:
X_train = X_train.astype('float32') / np.max(X_train)
X_val = X_val.astype('float32') / np.max(X_val)

In [294]:
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]

In [295]:
model = models.Sequential()
model.add(layers.Conv1D(64, kernel_size=2, activation='relu', padding='same', input_shape=(X_train.shape[1], 1)))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(128, kernel_size=2, activation='relu', padding='same'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Conv1D(256, kernel_size=2, activation='relu', padding='same'))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(2, activation='softmax'))

C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [296]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [297]:
model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_12 (Conv1D)                   │ (None, 27, 64)              │             192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_12 (MaxPooling1D)      │ (None, 13, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_13 (Conv1D)                   │ (None, 13, 128)             │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_13 (MaxPooling1D)      │ (None, 6, 128)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_14 (Conv1D)                   │ (None, 6, 256)              │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_14 (MaxPooling1D)      │ (None, 3, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ (None, 768)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_20 (Dense)                     │ (None, 512)                 │         393,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_22 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_23 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_24 (Dense)                     │ (None, 10)                  │             650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_25 (Dense)                     │ (None, 2)                   │              22 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 649,376 (2.48 MB)

 Trainable params: 649,376 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

In [298]:
batch_size = 128
epochs = 50

history = model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    verbose=1
)

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 227ms/step - accuracy: 0.5058 - loss: 0.6932 - val_accuracy: 0.4877 - val_loss: 0.6932
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.5048 - loss: 0.6919 - val_accuracy: 0.5333 - val_loss: 0.6930
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5801 - loss: 0.6917 - val_accuracy: 0.5509 - val_loss: 0.6919
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.5829 - loss: 0.6878 - val_accuracy: 0.5860 - val_loss: 0.6931
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5897 - loss: 0.6928 - val_accuracy: 0.5474 - val_loss: 0.6900
Epoch 6/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.5806 - loss: 0.6854 - val_accuracy: 0.5509 - val_loss: 0.6888
Epoch 7/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5917 - loss: 0.6806 - val_accuracy: 0.5649 - val_loss: 0.6858
Epoch 8/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.6195 - loss: 0.6738 - val_accuracy: 0.6281 - val_loss: 0.6773

In [299]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print("Validation Loss: ", val_loss)
print("Validation Accuracy: ", val_accuracy)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7291 - loss: 0.5588 
Validation Loss:  0.5739229917526245
Validation Accuracy:  0.7017543911933899


In [300]:
model.save('cnn_cough_classification.h5')

In [301]:
import pickle
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

with open('label_encoder_cnn_2.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

### PREDICTION

In [303]:
from tensorflow.keras.models import load_model

with open('label_encoder_cnn_2.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

model = load_model('cnn_cough_classification.h5')

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
audio_features = extract_features(new_audio_signal)

audio_features = np.array(audio_features).reshape(1, -1)  
audio_features_scaled = audio_features.astype('float32') / np.max(X_train)  
audio_features_scaled = audio_features_scaled[..., np.newaxis]

predictions = model.predict(audio_features_scaled)
predicted_label_index = np.argmax(predictions, axis=1)[0]
predicted_label = label_encoder.classes_[predicted_label_index]

print("Predicted Label: ",predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 414ms/step
Predicted Label:  non_cough


## DENSE NEURAL NETWORKS - 2

In [304]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

In [305]:
df = pd.read_csv("feature_extraction.csv")

In [307]:
X = df.drop(columns=['label']).values
y = df['label'].values

In [308]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [309]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [310]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [311]:
model = Sequential()

model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

C:\Users\pavan\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [312]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_26 (Dense)                     │ (None, 512)                 │          14,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_18 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_27 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_19 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_28 (Dense)                     │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 2)                   │             130 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 186,946 (730.26 KB)

 Trainable params: 186,946 (730.26 KB)

 Non-trainable params: 0 (0.00 B)

In [313]:
batch_size = 32
epochs = 50

history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(X_test, y_test))

Epoch 1/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - accuracy: 0.5227 - loss: 0.7157 - val_accuracy: 0.6647 - val_loss: 0.6231
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6187 - loss: 0.6354 - val_accuracy: 0.6821 - val_loss: 0.5607
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7346 - loss: 0.5436 - val_accuracy: 0.7341 - val_loss: 0.5179
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7877 - loss: 0.4529 - val_accuracy: 0.7861 - val_loss: 0.4694
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7666 - loss: 0.4766 - val_accuracy: 0.7746 - val_loss: 0.4431
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8098 - loss: 0.4332 - val_accuracy: 0.8150 - val_loss: 0.4322
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8143 - loss: 0.3815 - val_accuracy: 0.8035 - val_loss: 0.4254
Epoch 8/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8601 - loss: 0.3520 - val_accuracy: 0.8208 - v

In [314]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy: ",accuracy)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9286 - loss: 0.3178 
Test Accuracy:  0.9075144529342651


In [315]:
predictions = model.predict(X_test)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


In [316]:
model.save('dense_neural_network_model.h5')

### PREDICTION

In [322]:
import numpy as np
from tensorflow.keras.models import load_model

def extract_features_prediction(file_path):
    y, sr = librosa.load(file_path, sr=None)
    
    y, _ = librosa.effects.trim(y, top_db=20)
    
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))
    spectral_centroid = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
    spectral_bandwidth = np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
    spectral_contrast = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
    rms_energy = np.mean(librosa.feature.rms(y=y))
    
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=21)
    mfccs_mean = np.mean(mfccs, axis=1)
    
    features = np.hstack([zcr, spectral_centroid, spectral_bandwidth, spectral_contrast, spectral_rolloff, rms_energy, mfccs_mean])
    
    return features.astype(np.float32) 

model = load_model('dense_neural_network_model.h5')

def predict_audio(file_path, model):
    audio_features = extract_features(file_path)
    audio_features = audio_features.reshape(1, -1)  
    predictions = model.predict(audio_features)
    predicted_class = np.argmax(predictions, axis=1)[0]  
    
    return predicted_class

new_audio_signal = "H:/My Drive/Task 2 - Feature Extarction/kashish_mru.wav"
predicted_label = predict_audio(new_audio_signal, model)

if predicted_label == '1' :
    print("The audio signal contains cough")
else :
    print("The audio signal does not contains cough")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
The audio signal does not contains cough
